#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2018


# Homework 3:  Embeddings + Recommenders

### 100 points [5% of your final grade]

### Due: Monday, April 9 by 11:59pm

*Goals of this homework:* There are two main learning objectives: (i) implement and evaluate a pre-cursor to modern word2vec embeddings; and (ii) implement, evaluate, and improve upon traditional collaborative filtering recommenders.

*Submission Instructions:* To submit your homework, rename this notebook as UIN_hw#.ipynb. For example, this homework submission would be: YourUIN_hw3.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Thursday, April 12 at 11:59pm.

# Part 1: Word Embeddings (50 points)
For this first part, we're going to implement a word embedding approach that is a bit simpler than word2vec. The key idea is to look at co-occurrences between center words and context words (somewhat like in word2vec) but without any pesky learning of model parameters.

If you're interested in a deeper treatment of comparing count vs. learned embeddings, take a look at: [Don’t count, predict! A systematic comparison of
context-counting vs. context-predicting semantic vectors](
http://www.aclweb.org/anthology/P14-1023)

## Load the Brown Corpus

The dataset for this part is the (in)famous [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) that is a collection of text samples from a wide range of sources, with over one million unique words. Good for us, you can find the Brown corpus in nltk. *Make sure you have already installed nltk with something like: conda install nltk*

In [26]:
import nltk
from __future__ import print_function
nltk.download('brown')

[nltk_data] Downloading package brown to /Users/Yash/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

Once you have it locally, you can load the dataset into your notebook. You can access the words using brown.words():

In [27]:
from nltk.corpus import brown
brown.words()

[u'The', u'Fulton', u'County', u'Grand', u'Jury', ...]

## 1.1 Dataset Pre-processing
OK, now we need to do some basic pre-processing. For this part you should:

* Remove stopwords and punctuation.
* Make everything lowercase.

Then, count how often each word occurs. We will define the 5,000 most  frequent words as your vocabulary (V). We will define the 1,000 most frequent words as our context (C). Include a print statement below to show the top-20 words after pre-processing.

In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/Yash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
import string
from nltk.corpus import stopwords
stop = stopwords.words('english') + list(string.punctuation) + ["''", "'", '"', '""', '``', '`','--']

filtered_words = [word.lower() for word in brown.words() if not word.lower() in stop]
filtered_words

[u'fulton',
 u'county',
 u'grand',
 u'jury',
 u'said',
 u'friday',
 u'investigation',
 u"atlanta's",
 u'recent',
 u'primary',
 u'election',
 u'produced',
 u'evidence',
 u'irregularities',
 u'took',
 u'place',
 u'jury',
 u'said',
 u'term-end',
 u'presentments',
 u'city',
 u'executive',
 u'committee',
 u'over-all',
 u'charge',
 u'election',
 u'deserves',
 u'praise',
 u'thanks',
 u'city',
 u'atlanta',
 u'manner',
 u'election',
 u'conducted',
 u'september-october',
 u'term',
 u'jury',
 u'charged',
 u'fulton',
 u'superior',
 u'court',
 u'judge',
 u'durwood',
 u'pye',
 u'investigate',
 u'reports',
 u'possible',
 u'irregularities',
 u'hard-fought',
 u'primary',
 u'mayor-nominate',
 u'ivan',
 u'allen',
 u'jr.',
 u'relative',
 u'handful',
 u'reports',
 u'received',
 u'jury',
 u'said',
 u'considering',
 u'widespread',
 u'interest',
 u'election',
 u'number',
 u'voters',
 u'size',
 u'city',
 u'jury',
 u'said',
 u'find',
 u'many',
 u"georgia's",
 u'registration',
 u'election',
 u'laws',
 u'outmoded

In [30]:
from collections import defaultdict
word_dict = defaultdict(int)
for w in filtered_words:
    word_dict[w] += 1

In [31]:
import operator
sorted_dict_word = sorted(word_dict.items(), key=operator.itemgetter(1),reverse=True)
for i in range(20):
    print(sorted_dict_word[i])
vocab = [];context = []
i=0;j=0
for key in sorted_dict_word:
    if i<5000:
        vocab.append(key[0])
    if j<1000:
        context.append(key[0])
    i+=1
    j+=1     

(u'one', 3292)
(u'would', 2714)
(u'said', 1961)
(u'new', 1635)
(u'could', 1601)
(u'time', 1598)
(u'two', 1412)
(u'may', 1402)
(u'first', 1361)
(u'like', 1292)
(u'man', 1207)
(u'even', 1170)
(u'made', 1125)
(u'also', 1069)
(u'many', 1030)
(u'must', 1013)
(u'af', 996)
(u'back', 966)
(u'years', 950)
(u'much', 937)


In [32]:
vocab

[u'one',
 u'would',
 u'said',
 u'new',
 u'could',
 u'time',
 u'two',
 u'may',
 u'first',
 u'like',
 u'man',
 u'even',
 u'made',
 u'also',
 u'many',
 u'must',
 u'af',
 u'back',
 u'years',
 u'much',
 u'way',
 u'well',
 u'people',
 u'mr.',
 u'little',
 u'state',
 u'good',
 u'make',
 u'world',
 u'still',
 u'see',
 u'men',
 u'work',
 u'long',
 u'get',
 u'life',
 u'never',
 u'day',
 u'another',
 u'know',
 u'last',
 u'us',
 u'might',
 u'great',
 u'old',
 u'year',
 u'come',
 u'since',
 u'go',
 u'came',
 u'right',
 u'used',
 u'take',
 u'three',
 u'states',
 u'use',
 u'house',
 u'without',
 u'place',
 u'american',
 u'around',
 u'however',
 u'home',
 u'small',
 u'found',
 u'mrs.',
 u'1',
 u'thought',
 u'went',
 u'say',
 u'part',
 u'general',
 u'high',
 u'upon',
 u'school',
 u'every',
 u'got',
 u'united',
 u'left',
 u'number',
 u'course',
 u'war',
 u'always',
 u'away',
 u'something',
 u'fact',
 u'2',
 u'water',
 u'though',
 u'public',
 u'put',
 u'less',
 u'think',
 u'almost',
 u'hand',
 u'enough',

## 1.2 Building the Co-occurrence Matrix 

For each word in the vocabulary (w), we want to calculate how often context words from C appear in its surrounding window of size 4 (two words before and two words after).

In other words, we need to define a co-occurrence matrix that has a dimension of |V|x|C| such that each cell (w,c) represents the number of times c occurs in a window around w. 

In [33]:
# Your Code Here...
import numpy as np
from collections import Counter
co_mat = np.zeros((5000,1000))

for vi in range(len(vocab)):
    window = []
#     print(v)
    for i in range(len(filtered_words)):
        if filtered_words[i] == vocab[vi]:
            if i-1>=0:
                window.append(filtered_words[i-1])
            if i-2>=0:
                window.append(filtered_words[i-2])
            if i+1<len(filtered_words):
                window.append(filtered_words[i+1])
            if i+2<len(filtered_words):
                window.append(filtered_words[i+2])
#             print(vi,len(window))
#             print(i)
    word_intersection = set.intersection(set(window),set(context))
    window = [w for w in window if w in word_intersection]
    window_count = Counter(window)
    for wc in window_count:
        context_idx = context.index(wc)
        co_mat[vi, context_idx] = window_count[wc]
co_mat

array([[84., 76., 52., ...,  1.,  1.,  1.],
       [76., 70., 69., ...,  3.,  5.,  0.],
       [52., 69., 38., ..., 13.,  0.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 1.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.]])

## 1.3 Probability Distribution

Using the co-occurrence matrix, we can compute the probability distribution Pr(c|w) of context word c around w as well as the overall probability distribution of each context word c with Pr(c).  

In [34]:
prcw = np.zeros((5000,1000))
rsum = [sum(co_mat[i]) for i in range(5000)]
csum = [sum(co_mat[:,i]) for i in range(1000)]
for i in range(5000):
    for j in range(1000):
        prcw[i,j] = co_mat[i,j]/rsum[i]
        
pc = []
for i in range(1000):
    pc.append(csum[i]/sum(csum))
prcw

array([[0.0132346 , 0.01197416, 0.00819285, ..., 0.00015755, 0.00015755,
        0.00015755],
       [0.01459574, 0.01344344, 0.01325139, ..., 0.00057615, 0.00096025,
        0.        ],
       [0.01446453, 0.01919332, 0.01057024, ..., 0.00361613, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.02439024, ..., 0.        , 0.        ,
        0.        ],
       [0.03333333, 0.03333333, 0.03333333, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.02857143, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [35]:
pc

[0.013907577523381388,
 0.011643231003532326,
 0.008134463936055075,
 0.006977346834418622,
 0.007007833752066528,
 0.006996747600194562,
 0.005943563172357789,
 0.005990679317813645,
 0.0059200050996298615,
 0.005099629861104375,
 0.005153674851480209,
 0.004808618374465267,
 0.004664498400129708,
 0.004354086147714659,
 0.004427531903866434,
 0.0044386180557384,
 0.004258468087818952,
 0.004121276958403372,
 0.00428202616054688,
 0.004101876192627432,
 0.0038773816172201197,
 0.003856595082460183,
 0.003791463940212383,
 0.0031553959765583317,
 0.0035018382225572705,
 0.0035406397541091514,
 0.003532325140205177,
 0.0035517259059811173,
 0.003411763238597546,
 0.003263485957310001,
 0.003397905548757589,
 0.003260714419342009,
 0.003271800571213975,
 0.0031470813626543574,
 0.003396519779773593,
 0.0030750213754865783,
 0.003047305995806663,
 0.0029045717904551006,
 0.0028976429455351217,
 0.003098579448214506,
 0.002966931394734909,
 0.002964159856766918,
 0.002887942562647151,
 0.0

## 1.4 Embedding Representation

Now you can represent each vocabulary word as a |C| dimensional vector using this equation:

Vector(w)= max(0, log (Pr(c|w)/Pr(c)))

This is a traditional approach called *pointwise mutual information* that pre-dates word2vec by some time. 

In [36]:
import math
embed = np.zeros((5000,1000))
for i in range(5000):
    for j in range(1000):
        temp = prcw[i,j]/pc[j]
        if temp > 0:
            embed[i,j] = max(0, math.log(temp)) 
        else:
            embed[i,j] = 0
embed      

array([[0.        , 0.0280261 , 0.00715162, ..., 0.        , 0.        ,
        0.        ],
       [0.04829563, 0.14376639, 0.48799279, ..., 0.35678522, 0.99185802,
        0.        ],
       [0.03926589, 0.49983753, 0.26193233, ..., 2.19358217, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 1.09807337, ..., 0.        , 0.        ,
        0.        ],
       [0.87412406, 1.05183292, 1.41044805, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.89768224, 0.        , ..., 0.        , 0.        ,
        0.        ]])

## 1.5 Analysis

So now we have some embeddings for each word. But are they meaningful? For this part, you should:

- First, cluster the vocabulary into 100 clusters using k-means. Look over the words in each cluster, can you see any relation beween words? Discuss your observations.

- Second, for the top-20 most frequent words, find the nearest neighbors using cosine distance (1- cosine similarity). Do the findings make sense? Discuss.

In [37]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=100, random_state=0).fit(embed)

In [43]:
for i in range(100):
    cluster_index = kmeans.labels_==i
    if sum(cluster_index) > 30:
        print("cluster ", i)
        print(np.array(vocab)[cluster_index])

cluster  7
[u'york' u'club' u'played' u'chicago' u'park' u'post' u'opening' u'bank'
 u'trip' u'junior' u'virginia' u'village' u'san' u'league' u'paris'
 u'newspaper' u'california' u'guests' u'coast' u'boston' u'ended'
 u'streets' u'orchestra' u'songs' u'negroes' u'headed' u'roads' u'dallas'
 u'professor' u'bay' u'joined' u'motor' u'runs' u'extended' u'fort'
 u'games' u'divided' u'attend' u'faced' u'lake' u'largest' u'band'
 u'japanese' u'dozen' u'theater' u'la' u'gold' u'detective' u'indian'
 u'los' u'automobile' u'brilliant' u'welcome' u'massachusetts' u'mention'
 u'impressive' u'sports' u'hopes' u'grand' u'co.' u'1957' u'via'
 u'shooting' u'vacation' u'angeles' u'sold' u'lincoln' u'mail' u'boards'
 u'thousands' u'avenue' u'decade' u'georgia' u'native' u'ballet' u'cast'
 u'steel' u'pennsylvania' u'nearby' u'charges' u'arranged' u'pilot'
 u'manchester' u'acres' u'hundreds' u'mobile' u'agent' u'tour' u'heads'
 u'route' u"year's" u'mountains' u'ships' u'insisted' u'bureau' u'guns'
 u'jou

 u'improvements' u'enforcement' u'earnings']
cluster  29
[u'feed' u'daily' u'length' u'cattle' u'units' u'inches' u'100' u'gain'
 u'reduce' u'50' u'metal' u'animals' u'disease' u'efficiency' u'milk'
 u'mile' u'equivalent' u'salt' u'percentage' u'diameter' u'mounted'
 u'listed' u'pounds' u'continuous' u'seeds' u'reduction' u'improve'
 u'yield' u'fewer' u'beef' u'preceding' u'excessive' u'instances' u'cow'
 u'sewage' u'subsequent' u'tons' u'pound' u'prevention' u'aids' u'grain'
 u'released' u'cents' u'treat' u'drug' u'70' u'sheep' u'continuously'
 u'milligrams' u'28' u'conversion' u'75' u'nest' u'livestock' u'dairy'
 u'gains' u'exceed' u'lb.' u'caution' u'fever']
cluster  30
[u'time' u'know' u'suppose' u'sir' u'lord' u'names' u'spoke' u'captain'
 u'strange' u'team' u'drink' u'explained' u'lady' u'sam' u'looks' u'nice'
 u'memory' u'dog' u'murder' u'send' u'die' u'somehow' u'wine' u'everybody'
 u'smiled' u'beauty' u'song' u'sweet' u'wonder' u'answered' u'jury'
 u'aside' u'hanover' u'suppos

[u'would' u'first' u'made' u'well' u'make' u'work' u'another' u'might'
 u'take' u'every' u'set' u'find' u'give' u'second' u'done' u'help' u'gave'
 u'either' u'money' u'job' u'full' u'able' u'short' u'sound' u'play'
 u'says' u'longer' u'third' u'call' u'expected' u'return' u'stage'
 u'instead' u'added' u'beginning' u'fine' u'bring' u'paper' u'final'
 u'working' u'meet' u'decided' u'try' u'record' u'trial' u'list'
 u'continued' u'step' u'lead' u'press' u'note' u'immediately' u'charge'
 u'opportunity' u'decision' u'main' u'appear' u'account' u'ones'
 u'nuclear' u'letters' u'design' u'whatever' u'pool' u'wish' u'expect'
 u'continue' u'serve' u'easily' u'scene' u'write' u'remained' u'suggested'
 u'attack' u'reach' u'machine' u'original' u'race' u'exactly' u'unless'
 u'places' u'giving' u'usual' u'oil' u'worth' u'sign' u'project' u'caused'
 u'dance' u'pass' u'goes' u'cover' u'carry' u'add' u'break' u'check'
 u'battle' u'carefully' u'allowed' u'patient' u'build' u'otherwise'
 u'marked' u'lear

[u'one' u'fields' u'sharp' ... u'discharge' u'individually' u'insistence']


Clusters have words in similar context. For example, the cluster 7 has names of places Chicago, York, California, Boston.

In [129]:
freq_words = []

for i in range(20):
    freq_words.append(sorted_dict_word[i])
print(freq_words)

[(u'one', 3292), (u'would', 2714), (u'said', 1961), (u'new', 1635), (u'could', 1601), (u'time', 1598), (u'two', 1412), (u'may', 1402), (u'first', 1361), (u'like', 1292), (u'man', 1207), (u'even', 1170), (u'made', 1125), (u'also', 1069), (u'many', 1030), (u'must', 1013), (u'af', 996), (u'back', 966), (u'years', 950), (u'much', 937)]


In [135]:
import scipy
for word in freq_words:
    distance = []
    for i in range(len(vocab)):
        if vocab[i] == word[0]:
            index = i
            for row in range(5000):
                distance.append(scipy.spatial.distance.cosine(embed[row,:], embed[index, :]))
    
    closest_words = [x for y,x in sorted(zip(distance, vocab))]
    print (word, closest_words[1:10])

(u'one', 3292) [u'every', u'took', u'things', u'another', u'course', u'right', u'thing', u'perhaps', u'think']
(u'would', 2714) [u'could', u'might', u'want', u'must', u"can't", u'wanted', u'anything', u'think', u"i'll"]
(u'said', 1961) [u'know', u"i'm", u'think', u'tell', u'oh', u"i'll", u'asked', u'told', u'say']
(u'new', 1635) [u'york', u'city', u'development', u'local', u'system', u'modern', u'business', u'central', u'company']
(u'could', 1601) [u'would', u'might', u'want', u'wanted', u"i'll", u"can't", u'ever', u'going', u'knew']
(u'time', 1598) [u'day', u'long', u'play', u'since', u'short', u'moment', u'around', u'little', u'much']
(u'two', 1412) [u'three', u'four', u'several', u'six', u'many', u'five', u'couple', u'seven', u'ago']
(u'may', 1402) [u'also', u'might', u'would', u'shall', u'must', u'possible', u'likely', u'present', u'general']
(u'first', 1361) [u'second', u'last', u'next', u'every', u'later', u'took', u'early', u'four', u'final']
(u'like', 1292) [u'said', u'know', u

This shows the words similar meanings/context such as years, days, months.

# Part 2. Collaborative Filtering (50 points)

In this second part, you will implement collaborative filtering on the Netflix prize dataset -- don’t freak out, the provided sample dataset has only ~2000 items and ~28,000 users.

As background, read the paper [Empirical Analysis of Predictive Algorithms for Collaborative Filtering](https://arxiv.org/pdf/1301.7363.pdf) up to Section 2.1. Of course you can read further if you are interested, and you can also refer to the course slides for collaborative filtering.

## 2.1 Load Netflix Data

The dataset is subset of movie ratings data from the Netflix Prize Challenge. Download the dataset from Piazza. It contains a train set, test set, movie file, and README file. The last two files are original ones from the Netflix Prize, however; in this homework you will deal with train and test files which both are subsets of the Netflix training data. Each of train and test files has lines having this format: MovieID,UserID,Rating.

Your job is to predict a rating in the test set using those provided in the training set.

In [52]:
import pandas as pd
df_train = pd.read_csv("netflix-dataset/TrainingRatings.txt", header = None, names = ['movie', 'user', 'rating'])
df_test = pd.read_csv("netflix-dataset/TestingRatings.txt", header = None, names = ['movie', 'user', 'rating'])
print df_train.head()
print df_train.shape

   movie     user  rating
0      8  1744889     1.0
1      8  1395430     2.0
2      8  1205593     4.0
3      8  1488844     4.0
4      8  1447354     1.0
(3255352, 3)


In [1]:
import numpy as np
import pandas as pd
df_train = pd.read_csv("netflix-dataset/TrainingRatings.txt", header = None, names = ['movie', 'user', 'rating'])
df_test = pd.read_csv("netflix-dataset/TestingRatings.txt", header = None, names = ['movie', 'user', 'rating'])

user = np.array(range(27555))
item = np.array(range(1701))

userid = pd.DataFrame({'userid': user, "user": df_test.user.unique()})
itemid = pd.DataFrame({'movieid': item, "movie": df_test.movie.unique()})

df_test = pd.merge(df_test, userid, on='user')
df_test = pd.merge(df_test, itemid, on='movie')
df_test.head()

,movie,user,rating,userid,movieid
0,8,573364,1.0,0,0
1,8,2149668,3.0,1,0
2,8,1089184,3.0,2,0
3,8,2465894,3.0,3,0
4,8,534508,1.0,4,0


In [2]:
df_test_1000 = df_test[df_test.userid < 1000]

In [3]:
df_train = pd.merge(df_train, userid, on='user')
df_train = pd.merge(df_train, itemid, on='movie')

In [4]:
df_train.head()

,movie,user,rating,userid,movieid
0,8,1744889,1.0,3670,0
1,8,1395430,2.0,7280,0
2,8,1205593,4.0,11193,0
3,8,1488844,4.0,5033,0
4,8,1447354,1.0,24554,0


In [5]:
df_train.sort_values(by = 'userid').head()

,movie,user,rating,userid,movieid
1275158,11812,573364,5.0,0,1117
489491,4432,573364,3.0,0,431
528446,4640,573364,5.0,0,452
1071949,9689,573364,4.0,0,905
1089219,9728,573364,4.0,0,915


In [6]:
df_test.head()

,movie,user,rating,userid,movieid
0,8,573364,1.0,0,0
1,8,2149668,3.0,1,0
2,8,1089184,3.0,2,0
3,8,2465894,3.0,3,0
4,8,534508,1.0,4,0


In [95]:
user_dict = {}
movie_dict = {}

for row in df_train.itertuples():
    user_dict[row[2]]=row[4]
    movie_dict[row[1]]=row[5]

In [7]:

matrix=np.zeros((28978,1821))

for row in df_train.itertuples():
    matrix[row[4],row[5]] = row[3]
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:

matrix_test=np.zeros((27555,1701))

for row in df_test.itertuples():
    matrix_test[row[4],row[5]] = row[3]
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## 2.2 Implement CF

In this part, you will implement the basic collaborative filtering algorithm described in Section 2.1 of the paper -- that is, focus only on Equations 1 and 2 (where Equation 2 is just the Pearson correlation). You should consider the first 5,000 users with their associated items in the test set. 

Note that you should test the algorithm for a small set of users e.g., 10 users first and then run for 5,000 users. It may take long to run but you won't have memory issues. 

Set k to 0.1. 

In [9]:
mean = []
for i in range(len(matrix)):
    count = 0
    rsum = 0
    for j in range(len(matrix[0])):
        rsum += matrix[i][j]
        if matrix[i][j] != 0:
            count +=1
    mean.append(rsum/count)
len(mean)

/Users/Yash/Code/deep_learning/Anaconda/anaconda3/envs/cs670/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


28978

In [44]:
matrix_1000 = matrix[0:1000, ]

pear = pd.DataFrame(matrix_1000)
pear = pear.T
pear = pear.apply(pd.to_numeric)
pearson = pear.corr(method='pearson')
type(pearson)

pandas.core.frame.DataFrame

In [11]:
pearson

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.273249,0.265258,0.295960,0.317614,0.323929,0.165679,0.215183,0.268991,0.348761,...,0.394618,0.413352,0.236808,0.309251,0.319805,0.411317,0.226278,0.354865,0.385232,0.389578
1,0.273249,1.000000,0.245322,0.374570,0.221085,0.235981,0.268444,0.174810,0.259956,0.336352,...,0.204997,0.410889,0.165113,0.320682,0.288557,0.259773,0.208642,0.320348,0.313740,0.358782
2,0.265258,0.245322,1.000000,0.539751,0.207526,0.432011,0.319220,0.245242,0.269075,0.383203,...,0.278026,0.319739,0.302018,0.445975,0.374727,0.229516,0.240304,0.387334,0.399503,0.457577
3,0.295960,0.374570,0.539751,1.000000,0.315323,0.415849,0.443141,0.228820,0.248924,0.424537,...,0.286186,0.408726,0.425919,0.450647,0.467891,0.262173,0.243145,0.405165,0.466437,0.441899
4,0.317614,0.221085,0.207526,0.315323,1.000000,0.223071,0.193734,0.089932,0.241946,0.276767,...,0.228507,0.258578,0.253976,0.236210,0.311590,0.283175,0.190014,0.285755,0.261227,0.379280
5,0.323929,0.235981,0.432011,0.415849,0.223071,1.000000,0.303946,0.287727,0.346266,0.355167,...,0.280044,0.430862,0.393169,0.349683,0.528349,0.202396,0.249928,0.324042,0.289808,0.389334
6,0.165679,0.268444,0.319220,0.443141,0.193734,0.303946,1.000000,0.186719,0.208264,0.254298,...,0.200071,0.245680,0.273199,0.295809,0.334802,0.175308,0.181789,0.299511,0.272348,0.320672
7,0.215183,0.174810,0.245242,0.228820,0.089932,0.287727,0.186719,1.000000,0.205558,0.229180,...,0.345985,0.268136,0.186031,0.217021,0.248771,0.187848,0.272953,0.196201,0.276667,0.233855
8,0.268991,0.259956,0.269075,0.248924,0.241946,0.346266,0.208264,0.205558,1.000000,0.268377,...,0.249059,0.339341,0.327068,0.260833,0.434508,0.195112,0.250195,0.192649,0.250108,0.349256
9,0.348761,0.336352,0.383203,0.424537,0.276767,0.355167,0.254298,0.229180,0.268377,1.000000,...,0.333607,0.405747,0.294171,0.459923,0.406830,0.275644,0.269790,0.335313,0.461958,0.462397


In [12]:
type(pearson)
pearson = pearson.as_matrix()

In [13]:
pred_test = np.zeros((1000, 1701))
mean = np.array(mean)
mean = mean[0:1000]
mean.shape

(1000,)

In [21]:
for i in range(1000):
    mean_a = mean[i]
    active_user_pearson = np.array(pearson[i, :])
    s = np.nansum(np.absolute(active_user_pearson))
    if s == 0.0:
        s = 0.005
    for j in range(1701):
        if matrix_1000[i][j] != 0:
            item_ratings = np.array(matrix_1000[:, j], dtype='float')
            scores = np.array(item_ratings - mean) * active_user_pearson
            pred_test[i][j] = mean_a +(1/s)*np.nansum(scores)
            if i%100 ==0 and j%100==0:
                print(i, j, pred_test[i, j])
pred_test

(0, 400, 0.39078398803642944)
(200, 0, 0.48970202504727967)
(200, 600, 0.6445443559576542)
(300, 500, 0.5405276048376768)
(300, 700, 0.1397625317341995)
(500, 500, 1.1788754691261305)
(500, 800, 1.7286530623621075)
(500, 1000, 2.0279049001922202)
(600, 1400, 0.27343982308397763)
(700, 800, 1.4578473738495266)
(700, 1000, 1.7553338586407174)


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.23698306, 0.95995807, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

## 2.3 Evaluation 

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [22]:
# Mean Absolute Error
error = []
for i in range(1000):
    for j in range(1701):
        if matrix_test[i][j] != 0:
            error.append(abs(matrix_test[i][j]-pred_test[i][j]))
error = np.array(error, dtype='float')

In [23]:
np.mean(error)

3.385406367411509

In [24]:
# Root Mean Squared Error
np.sqrt(np.mean(np.square(error)))

3.5680896282207946

## 2.4 Extensions

Given your results in the previous part, can you do better? For this last part you should report on your best attempt at improving MAE and RMSE. Provide code, results, plus a brief discussion on your approach.

Error can be improved by using various other factors in collaborative filtering such as item-item collaboration, latent factors etc., 

*Insert discussion here*